In [34]:
def get_ml_result():
  load_dotenv()

  INFLUXDB_TOKEN = os.getenv("INFLUXDB_TOKEN")
  INFLUXDB_ORG = os.getenv("INFLUXDB_ORG")
  INFLUXDB_BUCKET = os.getenv("INFLUXDB_BUCKET")
  INFLUXDB_URL = os.getenv("INFLUXDB_URL")

  app = Flask(__name__)


  client = InfluxDBClient(url="http://localhost:8086", token=INFLUXDB_TOKEN, org=INFLUXDB_ORG, timeout=30_000)

  query_api = client.query_api()
  query = 'from(bucket: "crypto")\
    |> range(start: -1y)\
    |> filter(fn: (r) => r["_measurement"] == "BTCUSDT")'
  result = query_api.query(org=INFLUXDB_ORG, query=query)



  results = []
  for table in result:
    for record in table.records:
      results.append((record.get_field(), record.get_value(),record.get_time()))
      
  data=pd.DataFrame(results)
  data=data.rename({0:"_field",1:"_value",2:"_time"},axis=1)
  data['_time'] = pd.to_datetime(data['_time'],format='%Y-%m-%dT%H:%M:%SZ',errors='coerce')
  data['_value']=pd.to_numeric(data['_value'],errors='coerce',downcast="float")
  data.dropna(inplace=True)
  data=pd.pivot(data,index='_time',columns='_field',values='_value')
  data=data.reset_index()
  data
  exog=data.copy()
  exog.drop('high',axis=1,inplace=True)

  exog['_time']=pd.DatetimeIndex(exog.reset_index()['_time']) + pd.DateOffset(hours=1)
  data=data[1:]
  last_exog=exog.iloc[-1]
  exog=exog.iloc[:-1]

  data.set_index('_time',inplace=True)
  exog.set_index('_time',inplace=True)

  endog=data['high'].values.reshape(-1,1)
  model = sm.tsa.arima.ARIMA(endog=endog[:-1],exog=np.array(exog[:-1],dtype='float'), order=(5,3,3))
  results = model.fit(method='statespace')
  result=results.forecast(steps=1,exog=exog.iloc[-1])
  result=result[0]
  return result



In [35]:
import requests
from flask import Flask, jsonify #AGGIUNGERE A DOCKER

app = Flask(__name__)

@app.route("/planner_result")
def get_ml():
    c=0
    
    prediction = get_ml_result()

    url = "https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
    response = requests.get(url)
    data = response.json()
    last_price = float(data["price"])

    if last_price < prediction:
        c=1 #buy 
    else:
        c=0 #sell
    return jsonify({"prediction": prediction,"buy_sell": c})

if __name__ == "__main__":
    app.run(host="0.0.0.0",port="5010")



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5010
 * Running on http://192.168.0.100:5010
Press CTRL+C to quit
/Users/dariodm/opt/anaconda3/envs/proj_nuovo/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/dariodm/opt/anaconda3/envs/proj_nuovo/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
127.0.0.1 - - [06/Feb/2023 17:47:35] "GET /planner_result HTTP/1.1" 200 -


In [ ]:
get_ml()

NewConnectionError: <urllib3.connection.HTTPConnection object at 0x7fd19266b730>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known